
 SER5T67# Cyoda Client Demo

Welcome to the Cyoda Client Demo! This notebook demonstrates how to connect and interact with the Cyoda API. Follow the steps below to get started.

## Prerequisites

Before running the cells, ensure you have the following:
- Cyoda API credentials (API key, secret, etc.)
- Necessary Python packages installed

## Steps

1. **Setup**: Import required libraries and set up the environment.
2. **Authentication**: Authenticate with the Cyoda API.
3. **Basic Operations**: Perform basic operations using the API.
4. **Advanced Features**: Explore advanced features and functionalities.

Let's get started!


In [ ]:
%%script echo skipping
pip install -r ../requirements.txt

In [ ]:
# Setup environment variables
import os

API_KEY = os.environ["CYODA_API_KEY"]
API_SECRET = os.environ["CYODA_API_SECRET"]
API_URL = os.environ["CYODA_API_URL"]
GRPC_ADDRESS = os.environ["GRPC_ADDRESS"]
TOKEN = ""
print(API_KEY)
print(API_URL)
print(GRPC_ADDRESS)

In [ ]:
ENTITY_CLASS_NAME = "com.cyoda.tdb.model.treenode.TreeNodeEntity"
#increment the version before run
VERSION=612

In [ ]:
# Authenticate with the Cyoda API
import requests
import json

api_url = API_URL + "/api/auth/login"
headers = {"Content-Type": "application/json", "X-Requested-With": "XMLHttpRequest"}
auth_data = {"username": API_KEY, "password": API_SECRET}
print(api_url)
response = requests.post(api_url, headers=headers, data=json.dumps(auth_data))
print(response.json())
if response.status_code == 200:
    print("Authentication successful!")
    TOKEN = response.json().get("token")
else:
    print("Authentication failed. Please check your API credentials.")

In [ ]:
import requests
import logging
from typing import List
# Setup logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


In [ ]:
def send_get_request(path):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.get(url, headers=headers)
    return response

In [ ]:
def send_post_request(path, data):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.post(url, headers=headers, data=data)
    return response

In [ ]:
def send_put_request(path, data, timeout):
    url = f"{API_URL}/{path}"

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {TOKEN}"}
    response = requests.put(url, headers=headers, data=data, timeout=timeout)
    return response


In [ ]:
version = VERSION

In [ ]:
def save_entity_schema(model, version, data):
    path = f"api/treeNode/model/import/JSON/SAMPLE_DATA/{model}/{version}"
    response = send_post_request(path=path, data=data)
    logger.info(response)
    return response

In [ ]:
def test_save_employee_schema():
    model = "employee"
    employees = '[{"id": "9c5dcfd1-aec9-456a-820a-566a82269596", "fullName":"Russ Blick","department":"Legal"}]'
    employees_response = save_entity_schema(
        model=model, version=version, data=employees
    )
    logger.info(employees_response)
    assert (
        employees_response.status_code == 200
    ), f"Expected 200, got {employees_response.status_code}"


test_save_employee_schema()

curl -X POST \
  'https://API_URL/api/treeNode/model/import/JSON/SAMPLE_DATA/employee/213' \
  -H 'Content-Type: application/json' \
  -H 'Authorization: Bearer TOKEN' \
  -d '[{"id": "9c5dcfd1-aec9-456a-820a-566a82269596", "fullName": "Russ Blick", "department": "Legal"}]'

In [ ]:
def test_save_report_schema():
    model = "expense_report"
    reports = '[{"employeeId":"9c5dcfd1-aec9-456a-820a-566a82269596","city":"Alofi","departureDate":"2024-06-24T12:28:51.245+00:00","totalAmount":"515.38"},{"employeeId":"a50a7fbe-1e3b-11b2-9575-f2bfe09fbe21","city":"Muscat","departureDate":"2024-06-24T18:32:30.094+00:00","totalAmount":"318.59"}]'
    reports_response = save_entity_schema(model=model, version=version, data=reports)
    logger.info(reports_response)
    assert (
        reports_response.status_code == 200
    ), f"Expected 200, got {reports_response.status_code}"


test_save_report_schema()

In [ ]:
def test_save_payment_schema():
    model = "payment"
    payments = (
        '[{"btReportId":"a50a7fbe-1e3b-11b2-9575-f2bfe09fbe21","amount":"199.17"}]'
    )
    payment_response = save_entity_schema(model=model, version=version, data=payments)
    logger.info(payment_response.json())
    assert (
        payment_response.status_code == 200
    ), f"Expected 200, got {payment_response.status_code}"


test_save_payment_schema()

In [ ]:
def lock_entity_schema(model, version, data):
    path = f"api/treeNode/model/{model}/{version}/lock"
    response = send_put_request(path=path, data=data, timeout=None)
    logger.info(response)
    return response

In [ ]:
def test_lock_employee_schema():
    model = "employee"
    employees_response = lock_entity_schema(model=model, version=version, data=None)
    logger.info(employees_response)
    assert (
        employees_response.status_code == 200
    ), f"Expected 200, got {employees_response.status_code}"


test_lock_employee_schema()

In [ ]:
def test_lock_report_schema():
    model = "expense_report"
    report_response = lock_entity_schema(model=model, version=version, data=None)
    logger.info(report_response)
    assert (
        report_response.status_code == 200
    ), f"Expected 200, got {report_response.status_code}"


test_lock_report_schema()

In [ ]:
def test_lock_payment_schema():
    model = "payment"
    payment_response = lock_entity_schema(model=model, version=version, data=None)
    logger.info(payment_response)
    assert (
        payment_response.status_code == 200
    ), f"Expected 200, got {payment_response.status_code}"


test_lock_payment_schema()

In [ ]:
emplyee_ids=[]
report_ids=[]
payment_ids=[]

In [ ]:
def save_new_entity(model, version, data, entity_ids: List):
    path = f"api/entity/new/JSON/TREE/{model}/{version}"
    response = send_post_request(path=path, data=data)
    # Save entities ids for later use in the tests
    if response.status_code == 200:
        response_json = response.json()
        for item in response_json:
            item_ids = item['entityIds']
            entity_ids.extend(item_ids)
        logger.info(entity_ids)
    return response

In [ ]:
def test_save_new_employee():
    model = "employee"
    employees = '[{"id": "9c5dcfd1-aec9-456a-820a-566a82269596", "fullName":"Russ Blick","department":"Legal"}]'
    employees_response = save_new_entity(
        model=model, version=version, data=employees, entity_ids=emplyee_ids
    )
    logger.info(employees_response.json())
    assert (
        employees_response.status_code == 200
    ), f"Expected 200, got {employees_response.status_code}"

test_save_new_employee()

In [ ]:
def test_save_new_report():
    model = "expense_report"
    reports = '[{"employeeId":"9c5dcfd1-aec9-456a-820a-566a82269596","city":"Alofi","departureDate":"2024-06-24T12:28:51.245+00:00","totalAmount":"515.38"},{"employeeId":"a50a7fbe-1e3b-11b2-9575-f2bfe09fbe21","city":"Muscat","departureDate":"2024-06-24T18:32:30.094+00:00","totalAmount":"318.59"}]'
    reports_response = save_new_entity(
        model=model, version=version, data=reports, entity_ids=report_ids
    )
    logger.info(reports_response.json())
    assert (
        reports_response.status_code == 200
    ), f"Expected 200, got {reports_response.status_code}"


test_save_new_report()

In [ ]:
def test_save_payment_schema():
    model = "payment"
    payments = (
        '[{"btReportId":"a50a7fbe-1e3b-11b2-9575-f2bfe09fbe21","amount":"199.17"}]'
    )
    payment_response = save_new_entity(model=model, version=version, data=payments, entity_ids=payment_ids)
    logger.info(payment_response.json())
    assert (
        payment_response.status_code == 200
    ), f"Expected 200, got {payment_response.status_code}"


test_save_payment_schema()

In [ ]:
logger.info(emplyee_ids)
logger.info(report_ids)
logger.info(payment_ids)

In [ ]:
def get_entity_current_state(entityId):
    
    path = f"api/platform-api/entity-info/fetch/lazy?entityClass={ENTITY_CLASS_NAME}&entityId={entityId}&columnPath=state"
    response = send_get_request(path=path)
    logger.info(response)
    return response

curl -X GET \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer token" \
  "https://API_URL/api/platform-api/entity-info/fetch/lazy?entityClass=com.cyoda.tdb.model.treenode.TreeNodeEntity&entityId=d97c39e8-1dde-11b2-95c3-4a6f479a0680&columnPath=state"

In [ ]:
def launch_transition(entityId, transitionName):
    
    path = f"api/platform-api/entity/transition?entityId={entityId}&entityClass={ENTITY_CLASS_NAME}&transitionName={transitionName}"
    timeout = (30, 30)
    response = send_put_request(path=path, data=None, timeout = timeout)
    logger.info(response)
    return response

...

In [ ]:
# Step 1: Install gRPC and tools
!pip install grpcio grpcio-tools

# Step 2: Compile proto files
!python -m grpc_tools.protoc -I. --python_out=. --pyi_out=. --grpc_python_out=. cyoda-cloud-api.proto

!python -m grpc_tools.protoc -I. --python_out=. --pyi_out=. --grpc_python_out=. cloudevents.proto

In [ ]:
from enum import Enum


class CloudEventType(str, Enum):
    BASE_EVENT = "BaseEvent"
    CALCULATION_MEMBER_JOIN_EVENT = "CalculationMemberJoinEvent"
    CALCULATION_MEMBER_GREET_EVENT = "CalculationMemberGreetEvent"
    ENTITY_PROCESSOR_CALCULATION_REQUEST = "EntityProcessorCalculationRequest"
    ENTITY_PROCESSOR_CALCULATION_RESPONSE = "EntityProcessorCalculationResponse"

In [ ]:
from typing import Any, Optional
from pydantic import BaseModel


class DataPayload(BaseModel):
    type: str
    data: Optional[Any] = None

In [ ]:
from typing import Optional
from pydantic import BaseModel


class ErrorCode(BaseModel):
    code: str
    message: str


class BaseEvent(BaseModel):
    owner: str
    success: Optional[bool] = True
    error: Optional[ErrorCode] = None

In [ ]:
class CalculationMemberGreetEvent(BaseEvent):
    memberId: str

In [ ]:
from typing import List, Optional

class CalculationMemberJoinEvent(BaseEvent):
    tags: Optional[List[str]] = None

In [ ]:
class EntityProcessorCalculationRequest(BaseEvent):
    requestId: str
    entityId: str
    processorId: str
    processorName: str
    payload: DataPayload

In [ ]:
class EntityProcessorCalculationResponse(BaseEvent):
    requestId: str
    entityId: str
    payload: DataPayload

In [ ]:
# Step 3: Import the generated classes
import grpc
import cyoda_cloud_api_pb2_grpc
import cloudevents_pb2

channel = grpc.secure_channel(GRPC_ADDRESS, grpc.ssl_channel_credentials())
stub = cyoda_cloud_api_pb2_grpc.CloudEventsServiceStub(channel)
# Initialize the join event
join_event = CalculationMemberJoinEvent(owner="PLAY", tags=["simple", "sample"])


# Create a new CloudEvent
cloudEvent = cloudevents_pb2.CloudEvent()
cloudEvent.id = "9ba80b3e-e856-4bdb-984b-7523a458101b"
cloudEvent.source = "SimpleSample"
cloudEvent.spec_version = "1.0"
cloudEvent.type = "CalculationMemberJoinEvent"
print(join_event.model_dump_json())
cloudEvent.text_data = join_event.model_dump_json()  # Assign the serialized event

# Start streaming
responses = stub.startStreaming(iter([cloudEvent]))
for response in responses:
    print("Received event: ", response)